In [ ]:
from rat_trial import RatTrial
from utils import c3d_scan


c3d_dir = '/home/hudson/Downloads/TestDB'
unscaled_model = '/home/hudson/Rats_Python/opensim/Bilateral/FinalBilateral.osim'
marker_set = '/home/hudson/Rats_Python/opensim/Bilateral/bilateral_rodent_hindlimb_markerSet.xml'
ik_setup = '/home/hudson/Rats_Python/opensim/Bilateral/generic_Rat_Setup_IK.xml'
trc_filter_cutoff = 15 # Hz
ik_filter_cutoff = 15 # Hz

output_dir = '/home/hudson/Downloads/TestOut'

## Need to make sure that model gets scaled first before doing anything else
# Sorting alphabetically makes sure that directories are done together and static trial is first
c3d_files = sorted(c3d_scan(c3d_dir)) 
print(f'Found {len(c3d_files)} C3D files.')
model_file = None
for c3d_file in c3d_files:
    trial = RatTrial.from_c3d_file(c3d_file)

    if not trial.validate_trial():
        continue
    print(f'Processing {trial.classification}/{trial.subject_names}/{trial.session_name}/{trial.name}...')
    trc_file = trial.to_trc()
    # If static
    if trial.trial_type == 'Static':
        # Scale model
        model_file = trial.create_scaled_model()
    # Else
    else:
        # IK - filter TRC data
        trial.inverse_kinematics()
        # ID
        trial.inverse_dynamics()

2025-06-09 06:56:03.456 | INFO     | rat_trial:valid_static:69 - Trial Static01 is a valid static trial


Found 203 C3D files.
Processing BAA/['BAA01']/Baseline/Static01...


TypeError: Trial.to_trc() missing 1 required positional argument: 'filepath'

In [ ]:
# Define groups
NR = []
TEMR = []
HH = []
TEMR_HH = []
KG = []
HS = []
TEMR_KG = []



